In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.append(os.path.join(os.getcwd(), 'src'))

In [2]:
from slack_interactor import SlackInteractor
from claude_llm import ClaudeLLM
from db import ActionDatabase
from config import CONFIG
from agent_interface import BaseAgent
from project_manager_agent import ProjectManagerAgent
from sarcastic_agent import SarcasticAgent
from paul_graham_agent import PaulGrahamAgent  # Add this import
from drunk_agent import DrunkAgent
from runner import process_threads, execute_due_actions

In [3]:
workspaces = CONFIG['workspaces']
slack_interactors = {
    workspace_config['name']: SlackInteractor(workspace_config)
    for workspace_config in workspaces
}

agents = {}
for workspace_config in workspaces:
    workspace_name = workspace_config['name']
    slack_interactor = slack_interactors[workspace_name]
    action_db = ActionDatabase(workspace_name)
    agents[workspace_name] = []

    agent_classes = {
        'ProjectManagerAgent': ProjectManagerAgent,
        'SarcasticAgent': SarcasticAgent,
        'PaulGrahamAgent': PaulGrahamAgent,
        'DrunkAgent': DrunkAgent
    }

    for agent_config in workspace_config.get('agents', []):
        agent_name = agent_config['name']
        llm_type = agent_config['llm_type']
        if agent_name in agent_classes:
            agent_class = agent_classes[agent_name]
            agent = agent_class(llm_type, action_db, slack_interactor, workspace_name=workspace_name)
            agents[workspace_name].append(agent)
        else:
            print(f"Warning: Unknown agent type '{agent_name}' for workspace '{workspace_name}'")

In [4]:
for workspace_name, slack_interactor in slack_interactors.items():
    print(f"\nFetching new messages for workspace: {workspace_name}")
    data = slack_interactor.fetch_new_user_messages()
    threads = slack_interactor.organize_threads(data)
    print(f"Found {len(threads)} threads with new user messages in {workspace_name}.")

    if not slack_interactor.is_first_run:
        results = process_threads(agents[workspace_name], threads)
        print(f"\nChecking for due actions in {workspace_name}...")
        execute_due_actions(agents[workspace_name])
    else:
        print(f"First run for {workspace_name}. Skipping thread processing and due actions.")
        slack_interactor.is_first_run = False


Fetching new messages for workspace: agentflow
Set conversations_oldest to 1722399936.894109
Loaded 611 old messages
Found 2 new messages
Updated complete_conversations_agentflow.pkl with new messages
Found 0 threads with new user messages in agentflow.

Checking for due actions in agentflow...

Executing delayed action for Sarcastic Agent in thread: 2024-08-31 02:19:02.751599073
Action: Check in with Eli about any modifications he wants for Tipsy Agent to make the interactions more fun and engaging.
OpenAI gpt-4o prompt:

        You are an AI agent bot named Agentflow with the following characteristics:
        Personality: Funny and sarcastic
        Goal: Intermittently inject sarcasm to conversations. Be sparse in responses to avoid being annoying.
        Username: Sarcastic Agent
Due task to execute: Check in with Eli about any modifications he wants for Tipsy Agent to make the interactions more fun and engaging.

        Analyze the following conversation carefully. Consider t

Posted reply to thread 2024-08-31 02:19:02.751599073 in channel agentflow
Posted response in thread: 2024-08-31 02:19:02.751599073
Removed executed action from database

Fetching new messages for workspace: legal flow
Set conversations_oldest to 1724897852.811579
Loaded 29 old messages
Found 0 new messages
Updated complete_conversations_legal flow.pkl with new messages
Found 0 threads with new user messages in legal flow.

Checking for due actions in legal flow...

Fetching new messages for workspace: $10M or bust
Set conversations_oldest to 1725401131.040029
Loaded 3 old messages
Found 0 new messages
Updated complete_conversations_$10M or bust.pkl with new messages
Found 0 threads with new user messages in $10M or bust.

Checking for due actions in $10M or bust...
